In [ ]:
from huggingface_hub import hf_hub_download
hf= hf_hub_download(repo_id='ai-forever/Real-ESRGAN', filename='RealESRGAN_x2.pth')
print(hf)

In [ ]:
im_path='/opt/stash/stash-ai-helper/local_assets/performer_611/d488a2b0-a2bf-4f07-94e0-a3b38c30353c.JPEG'


In [ ]:
import IPython.core.display as display
from PIL import Image

In [ ]:
im= Image.open(im_path)
display.display(im)

In [ ]:
!pip install git+https://github.com/sberbank-ai/Real-ESRGAN.git
!pip install nvidia-cudnn-cu12~=9.3

In [ ]:
!pip install huggingface_hub==0.25

In [ ]:
import torch
from PIL import Image
import numpy as np
from RealESRGAN import RealESRGAN

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = RealESRGAN(device, scale=2)
model.load_weights('weights/RealESRGAN_x4.pth', download=True)

In [ ]:
sr_image = model.predict(im)
display.display(sr_image)

In [ ]:
from transformers import pipeline
#upscaler= pipeline("image-to-image", model='ai-forever/Real-ESRGAN', filename='RealESRGAN_x2.pth')
from transformers import AutoModelForImageToImage
#hf= hf_hub_download(repo_id='ai-forever/Real-ESRGAN', filename='RealESRGAN_x2.pth')
ai= AutoModelForImageToImage.from_pretrained(pretrained_model_name_or_path=hf)


In [ ]:
import tensorflow as tf
from tensorflow import keras
import torch

In [ ]:
# Install basicsr - https://github.com/xinntao/BasicSR
# We use BasicSR for both training and inference
!pip install basicsr

# Install facexlib - https://github.com/xinntao/facexlib
# We use face detection and face restoration helper in the facexlib package
!pip install facexlib


# If you want to enhance the background (non-face) regions with Real-ESRGAN,
# you also need to install the realesrgan package
!pip install realesrgan
!pip install nvidia-cudnn-cu12~=9.3

In [ ]:
!pip install -r ../requirements.txt

In [ ]:
import sys
import torchvision.transforms.functional as functional
sys.modules["torchvision.transforms.functional_tensor"] = functional
from basicsr.archs.srvgg_arch import SRVGGNetCompact
from realesrgan.utils import RealESRGANer

sr_model = SRVGGNetCompact(num_in_ch=3, num_out_ch=3, num_feat=64, num_conv=32, upscale=4, act_type='prelu')
half = True if torch.cuda.is_available() else False
realesrganer = RealESRGANer(scale=4, model_path=hf, model=sr_model, tile=0, tile_pad=10, pre_pad=0, half=half)

# Function to upscale image with RealESRGAN
def upscale_image(image_path, output_path):
    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    # Upscale the image with RealESRGAN
    output, _ = realesrganer.enhance(img, outscale=4)
    cv2.imwrite(output_path, output)
    return output

In [ ]:
hr_image = tf.image.decode_image(tf.io.read_file(im_path))
# If PNG, remove the alpha channel. The model only supports
# images with 3 color channels.
if hr_image.shape[-1] == 4:
    hr_image = hr_image[...,:-1]
hr_size = (tf.convert_to_tensor(hr_image.shape[:-1]) // 4) * 4
hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
hr_image = tf.cast(hr_image, tf.float32)    
pre_img= tf.expand_dims(hr_image, 0)

In [ ]:
type(pre_img)

In [ ]:
!pip install tensorflow_hub

In [ ]:
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.metrics.SparseCategoricalAccuracy()])

  return model
model = create_model()


In [ ]:
#model.load_weights(hf)
#print(hf)
checkpoint= torch.load(hf)

In [ ]:
checkpoint.eval()